In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

In [2]:
def score(reduc_time, accuracy, train_time, red_w = 200, ac_w = 1, train_w = 20):
    return (accuracy ** ac_w)*(red_w/int(reduc_time + 1) + train_w/int(train_time + 1))

def analyse(path, reduction_weight, accuracy_weight, training_weight):
    df_each = []
    for filename in os.listdir(path):
        f = os.path.join(path, filename)
        if os.path.isfile(f) and ".DS" not in f:
            temp = pd.read_csv(f)
            temp = temp[temp.name.apply(lambda x: x != "name")]
            df_each.append((temp, filename))

    included_cols = ["name", "filename", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]

    out_df = pd.DataFrame(columns=included_cols)
    collected_all_df = pd.DataFrame(columns=included_cols)

    for res in df_each:
        res_copy = res[0].copy()
        res_copy["filename"] = res[1]
        score_series = res_copy[res_copy.name != "Nothing"][["reduction_time", "train_time", "accuracy"]].apply(lambda x: score(x["reduction_time"], x["accuracy"], reduction_weight, accuracy_weight, training_weight), axis=1)
        res_copy_score = res_copy.copy()
        res_copy_score["score_series"] = score_series
        out_df = pd.concat([out_df, res_copy_score])
        max_ind = res_copy_score.groupby(by=["original_shape"])["score_series"].idxmax()
        collected = res_copy_score.iloc[max_ind][["name", "filename", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]]
        collected_all_df = pd.concat([collected_all_df, collected])

    all = out_df.groupby(by="filename", group_keys=True).apply(lambda x: x[:])
    best = collected_all_df.groupby(["filename", "name"]).count().sort_values(by="original_shape").groupby(level=0).tail(1).sort_values(by="filename")

    return all, best

In [3]:
reduction_weight = 10
accuracy_weight = 10
training_weight = 10

In [4]:
path = "../output/kmeans/"
kmeans_all, kmeans_best = analyse(path, reduction_weight, accuracy_weight, training_weight)

In [5]:
kmeans_best

,,original_shape,transformed_shape,params,reduction_time,accuracy,train_time,score_series
filename,name,,,,,,,
"1000_0.33_(-100, 100)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.33_(-1000, 1000)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.33_(-500, 500)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.66_(-100, 100)",extremely sparse JL transform,6,6,6,6,6,6,6
"1000_0.66_(-1000, 1000)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.66_(-500, 500)",extremely sparse JL transform,4,4,4,4,4,4,4
"1000_0.99_(-100, 100)",extremely sparse JL transform,4,4,4,4,4,4,4
"1000_0.99_(-1000, 1000)",JL transform,5,5,5,5,5,5,5
"1000_0.99_(-500, 500)",extremely sparse JL transform,3,3,3,3,3,3,3


In [6]:
path = "../output/news/"
news_all, news_best = analyse(path, reduction_weight, accuracy_weight, training_weight)

In [12]:
kmeans_all

name  \
filename                                                   
1000_0.33_(-100, 100) 0    extremely sparse JL transform   
                      1    extremely sparse JL transform   
                      2    extremely sparse JL transform   
                      3    extremely sparse JL transform   
                      4    extremely sparse JL transform   
...                                                  ...   
500_0_(-500, 500)     255                   JL transform   
                      256                   JL transform   
                      257                   JL transform   
                      258                            PCA   
                      259                        Nothing   

                                        filename  original_shape  \
filename                                                           
1000_0.33_(-100, 100) 0    1000_0.33_(-100, 100)    (1000, 1000)   
                      1    1000_0.33_(-100, 100)    (1000, 1000)   
                      2    1000_0.33_(-100, 100)    (1000, 1000)   
                      3    1000_0.33_(-100, 100)    (1000, 1000)   
                      4    1000_0.33_(-100, 100)    (1000, 1000)   
...                                          ...             ...   
500_0_(-500, 500)     255      500_0_(-500, 500)  (20000, 20000)   
                      256      500_0_(-500, 500)  (20000, 20000)   
                      257      500_0_(-500, 500)  (20000, 20000)   
                      258      500_0_(-500, 500)  (20000, 20000)   
                      259      500_0_(-500, 500)  (20000, 20000)   

                          transformed_shape  \
filename                                      
1000_0.33_(-100, 100) 0        (1110, 2351)   
                      1        (1110, 2255)   
                      2        (1110, 1175)   
                      3        (1110, 1127)   
                      4         (1110, 235)   
...                                     ...   
500_0_(-500, 500)     255      (20110, 221)   
                      256       (20110, 88)   
                      257       (20110, 68)   
                      258      (20110, 201)   
                      259    (20110, 20000)   

                                                                params  \
filename                                                                 
1000_0.33_(-100, 100) 0                       {'ep': 0.05, 'de': 0.05}   
                      1                        {'ep': 0.05, 'de': 0.1}   
                      2                        {'ep': 0.1, 'de': 0.05}   
                      3                         {'ep': 0.1, 'de': 0.1}   
                      4                        {'ep': 0.5, 'de': 0.05}   
...                                                                ...   
500_0_(-500, 500)     255                       {'ep': 0.5, 'de': 0.1}   
                      256                      {'ep': 0.9, 'de': 0.05}   
                      257                       {'ep': 0.9, 'de': 0.1}   
                      258  {'n_components': 201, 'svd_solver': 'auto'}   
                      259                                           {}   

                           reduction_time  accuracy  train_time  score_series  \
filename                                                                        
1000_0.33_(-100, 100) 0          0.016116      0.97    0.638783  8.715012e+00   
                      1          0.036851      0.43    0.621037  2.554084e-03   
                      2          0.022616      0.58    0.458997  5.091322e-02   
                      3          0.012347      0.63    0.319385  1.164009e-01   
                      4          0.008589      0.16    0.259311  1.299423e-07   
...                                   ...       ...         ...           ...   
500_0_(-500, 500)     255        2.269467      1.00    0.362224  5.151515e+00   
                      256        1.476547      0.97    0.227300  5.027892e+00   
 

In [7]:
news_best

,,original_shape,transformed_shape,params,reduction_time,accuracy,train_time,score_series
filename,name,,,,,,,
2,extremely sparse JL transform,1,1,1,1,1,1,1
3,extremely sparse JL transform,1,1,1,1,1,1,1
5,extremely sparse JL transform,1,1,1,1,1,1,1


In [8]:
path = "../output/lin_reg/"

In [9]:
reg_all, reg_best = analyse(path, reduction_weight, accuracy_weight, training_weight)

In [10]:
reg_all.loc["1000_0"]

,name,filename,original_shape,transformed_shape,params,reduction_time,accuracy,train_time,score_series,run_num,...,original_std_max,original_std_min,original_sparsity,transformed_std_sum,transformed_std_mean,transformed_std_median,transformed_std_max,transformed_std_min,transformed_sparsity,characteristics
0,extremely sparse JL transform,1000_0,"(800, 999)","(1000, 2322)","{'ep': 0.05, 'de': 0.05}",0.081309,0.991170,0.634847,1.081515e+01,0.0,...,153814.246660,53734.561934,0.0,6.185990e+07,6.185990e+04,6.009268e+04,8.898140e+04,33624.755765,0.0,"{'n': 1000, 'd': 1000, 'x_range': 10000, 'coef..."
1,extremely sparse JL transform,1000_0,"(800, 999)","(1000, 2226)","{'ep': 0.05, 'de': 0.1}",0.018250,0.973709,0.482785,9.054006e+00,0.0,...,153814.246660,53734.561934,0.0,5.224789e+07,5.224789e+04,5.228416e+04,6.815595e+04,33609.354230,0.0,"{'n': 1000, 'd': 1000, 'x_range': 10000, 'coef..."
2,extremely sparse JL transform,1000_0,"(800, 999)","(1000, 1161)","{'ep': 0.1, 'de': 0.05}",0.034796,0.766579,0.354266,8.281655e-01,0.0,...,153814.246660,53734.561934,0.0,1.395197e+08,1.395197e+05,1.343591e+05,2.323182e+05,50924.808713,0.0,"{'n': 1000, 'd': 1000, 'x_range': 10000, 'coef..."
3,extremely sparse JL transform,1000_0,"(800, 999)","(1000, 1113)","{'ep': 0.1, 'de': 0.1}",0.011325,0.181235,0.293531,4.518333e-07,0.0,...,153814.246660,53734.561934,0.0,8.852732e+07,8.852732e+04,8.607918e+04,1.330149e+05,48303.996614,0.0,"{'n': 1000, 'd': 1000, 'x_range': 10000, 'coef..."
4,extremely sparse JL transform,1000_0,"(800, 999)","(1000, 232)","{'ep': 0.5, 'de': 0.05}",0.007037,0.655796,0.025002,1.738758e-01,0.0,...,153814.246660,53734.561934,0.0,3.396663e+08,3.396663e+05,3.310628e+05,5.922629e+05,75558.109383,0.0,"{'n': 1000, 'd': 1000, 'x_range': 10000, 'coef..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,JL transform,1000_0,"(16000, 9999)","(20000, 221)","{'ep': 0.5, 'de': 0.1}",1.607363,0.877256,0.218929,1.840488e+00,0.0,...,224458.311993,99529.900779,0.0,2.134304e+10,1.067152e+06,1.067538e+06,1.530444e+06,616712.864739,0.0,"{'n': 20000, 'd': 10000, 'x_range': 10000, 'co..."
204,JL transform,1000_0,"(16000, 9999)","(20000, 88)","{'ep': 0.9, 'de': 0.05}",1.630216,0.736913,0.144630,3.219803e-01,0.0,...,224458.311993,99529.900779,0.0,3.482116e+10,1.741058e+06,1.734343e+06,2.665841e+06,968958.014219,0.0,"{'n': 20000, 'd': 10000, 'x_range': 10000, 'co..."
205,JL transform,1000_0,"(16000, 9999)","(20000, 68)","{'ep': 0.9, 'de': 0.1}",0.816336,0.717685,0.079260,4.284379e-01,0.0,...,224458.311993,99529.900779,0.0,3.831971e+10,1.915986e+06,1.909033e+06,3.023940e+06,981091.407398,0.0,"{'n': 20000, 'd': 10000, 'x_range': 10000, 'co..."
206,PCA,1000_0,"(16000, 9999)","(20000, 200)","{'n_components': 200, 'svd_solver': 'auto'}",36.636848,0.935635,0.316000,1.073712e+00,0.0,...,224458.311993,99529.900779,0.0,2.213328e+10,1.106664e+06,1.108816e+06,1.569929e+06,656056.444796,0.0,"{'n': 20000, 'd': 10000, 'x_range': 10000, 'co..."


In [11]:
reg_best

,,original_shape,transformed_shape,params,reduction_time,accuracy,train_time,score_series
filename,name,,,,,,,
1000_0,extremely sparse JL transform,4,4,4,4,4,4,4
1000_0.1,extremely sparse JL transform,5,5,5,5,5,5,5
1000_0.3,extremely sparse JL transform,4,4,4,4,4,4,4
1000_0.5,extremely sparse JL transform,5,5,5,5,5,5,5
1000_0.9,extremely sparse JL transform,5,5,5,5,5,5,5
100_0,extremely sparse JL transform,4,4,4,4,4,4,4
100_0.1,extremely sparse JL transform,4,4,4,4,4,4,4
100_0.3,extremely sparse JL transform,5,5,5,5,5,5,5
100_0.5,extremely sparse JL transform,5,5,5,5,5,5,5
